In [1]:
# Import dependancies
import requests
import json
import pandas as pd
from pprint import pprint
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
import joblib
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from keras.models import load_model
import keras

In [2]:
# Create an API request using the link provided by asterank to gather data
url = "http://asterank.com/api/asterank?query=" 
response = requests.get(url + '{"price":{"$lt":1, "$gt":0}}&limit=1000').json()
# Data retrieval is limited to 1000 rows by default and can not be changed (by design)

# Create dataframe using data retrieved by the request above
df = pd.DataFrame(response)
df = df.sort_values(by = 'price' , ascending = False)

In [3]:
# Second request to get more data into the final dataframe
response2 = requests.get(url + '{"price":{"$gt":1}}&limit=1000').json()
df2 = pd.DataFrame(response2)

In [4]:
# Using only colummns we felt were necessary for our project
neo_df = df[['full_name', 'spec' , 'class' , 'ad' , 'q' , 'a' , 'e' , 'dv' ,'per' , 'price' , 'profit' , 'moid' , 'diameter']]
neo_df2 = df2[['full_name', 'spec' , 'class' , 'ad' , 'q' , 'a' , 'e' , 'dv' ,'per' , 'price' , 'profit' , 'moid' , 'diameter']]
neo_df2

,full_name,spec,class,ad,q,a,e,dv,per,price,profit,moid,diameter
0,(2002 AT4),D,AMO,2.700043,1.033903,1.866973,0.446214,5.557219,931.763080,1.599387e+03,3.344928e+02,0.042987,
1,(2001 SG286),D,APO,1.829700,0.886638,1.358169,0.347181,5.640122,578.134644,2.420770e+03,4.868816e+02,0.005336,
2,(2001 YE1),T,APO,2.870736,0.954085,1.912410,0.501109,5.905332,965.984260,1.450130e+04,2.547227e+03,0.059394,
3,(2001 UU92),T,AMO,5.287454,1.053954,3.170704,0.667596,7.286746,2062.205995,3.814231e+04,4.300143e+03,0.118818,
4,96 Aegle,T,MBA,3.479834,2.621945,3.050889,0.140597,NaN,1946.427065,6.196000e+04,3.981251e+03,1.641370,177.774
...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,372 Palma (1893 AH),B,MBA,3.966001,2.334280,3.150141,0.258992,11.987880,2042.176736,4.065526e+18,2.615080e+17,1.446870,173.642
992,423 Diotima (1896 DB),C,MBA,3.176470,2.956412,3.066441,0.035882,11.224330,1961.328629,4.235026e+18,2.908848e+17,1.942490,175.859
993,702 Alauda (1910 KQ),B,MBA,3.247375,3.137278,3.192327,0.017244,12.757615,2083.336507,5.408989e+18,3.264933e+17,2.120800,190.98
994,334 Chicago (1892 L),C,OMB,3.981728,3.798870,3.890299,0.023502,11.703702,2802.676341,6.115258e+18,4.009727e+17,2.782950,198.77


## Headers

spec = spectral type

ad = aphelion distance (AU)

q = perihelion (AU)

a = semi-major axis (AU) => (longest diameter from its obit)

e = eccentricity

dv = delta-velocity

per = period => (how long for 1 revolution around the sun)

moid = minimum orbit intersection distance

In [5]:
# Merging both datasets by appending
merged_df = neo_df.append(neo_df2)
merged_df

,full_name,spec,class,ad,q,a,e,dv,per,price,profit,moid,diameter
797,532 Herculina (1904 NY),S,MBA,3.258720,2.286373,2.772546,0.175353,10.733275,1686.229234,8.378840e-37,6.035861e-38,1.35440,167.791
796,354 Eleonora (1893 A),Sl,MBA,3.117824,2.479441,2.798633,0.114053,11.416972,1710.083231,5.863736e-37,3.967666e-38,1.49290,148.97
795,471 Papagena (1901 GN),S,MBA,3.555410,2.232245,2.893827,0.228619,10.449882,1798.072857,5.764868e-37,4.264148e-38,1.26726,148.128
794,349 Dembowska (1892 T),R,MBA,3.190289,2.654931,2.922610,0.091589,10.493194,1824.965628,4.843060e-37,3.564270e-38,1.65738,139.77
793,40 Harmonia,S,MBA,2.373855,2.160638,2.267247,0.047021,NaN,1246.942831,2.442248e-37,1.579404e-38,1.16159,111.251
...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,372 Palma (1893 AH),B,MBA,3.966001,2.334280,3.150141,0.258992,11.987880,2042.176736,4.065526e+18,2.615080e+17,1.44687,173.642
992,423 Diotima (1896 DB),C,MBA,3.176470,2.956412,3.066441,0.035882,11.224330,1961.328629,4.235026e+18,2.908848e+17,1.94249,175.859
993,702 Alauda (1910 KQ),B,MBA,3.247375,3.137278,3.192327,0.017244,12.757615,2083.336507,5.408989e+18,3.264933e+17,2.12080,190.98
994,334 Chicago (1892 L),C,OMB,3.981728,3.798870,3.890299,0.023502,11.703702,2802.676341,6.115258e+18,4.009727e+17,2.78295,198.77


In [6]:
# Replacing empty cells with 'nan' to be able to drop eaier later
new_df = merged_df.replace('', np.nan, inplace=False)
new_df

,full_name,spec,class,ad,q,a,e,dv,per,price,profit,moid,diameter
797,532 Herculina (1904 NY),S,MBA,3.258720,2.286373,2.772546,0.175353,10.733275,1686.229234,8.378840e-37,6.035861e-38,1.35440,167.791
796,354 Eleonora (1893 A),Sl,MBA,3.117824,2.479441,2.798633,0.114053,11.416972,1710.083231,5.863736e-37,3.967666e-38,1.49290,148.97
795,471 Papagena (1901 GN),S,MBA,3.555410,2.232245,2.893827,0.228619,10.449882,1798.072857,5.764868e-37,4.264148e-38,1.26726,148.128
794,349 Dembowska (1892 T),R,MBA,3.190289,2.654931,2.922610,0.091589,10.493194,1824.965628,4.843060e-37,3.564270e-38,1.65738,139.77
793,40 Harmonia,S,MBA,2.373855,2.160638,2.267247,0.047021,NaN,1246.942831,2.442248e-37,1.579404e-38,1.16159,111.251
...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,372 Palma (1893 AH),B,MBA,3.966001,2.334280,3.150141,0.258992,11.987880,2042.176736,4.065526e+18,2.615080e+17,1.44687,173.642
992,423 Diotima (1896 DB),C,MBA,3.176470,2.956412,3.066441,0.035882,11.224330,1961.328629,4.235026e+18,2.908848e+17,1.94249,175.859
993,702 Alauda (1910 KQ),B,MBA,3.247375,3.137278,3.192327,0.017244,12.757615,2083.336507,5.408989e+18,3.264933e+17,2.12080,190.98
994,334 Chicago (1892 L),C,OMB,3.981728,3.798870,3.890299,0.023502,11.703702,2802.676341,6.115258e+18,4.009727e+17,2.78295,198.77


In [7]:
# Dropping all null values
new_df = new_df.dropna()
new_df

,full_name,spec,class,ad,q,a,e,dv,per,price,profit,moid,diameter
797,532 Herculina (1904 NY),S,MBA,3.258720,2.286373,2.772546,0.175353,10.733275,1686.229234,8.378840e-37,6.035861e-38,1.35440,167.791
796,354 Eleonora (1893 A),Sl,MBA,3.117824,2.479441,2.798633,0.114053,11.416972,1710.083231,5.863736e-37,3.967666e-38,1.49290,148.97
795,471 Papagena (1901 GN),S,MBA,3.555410,2.232245,2.893827,0.228619,10.449882,1798.072857,5.764868e-37,4.264148e-38,1.26726,148.128
794,349 Dembowska (1892 T),R,MBA,3.190289,2.654931,2.922610,0.091589,10.493194,1824.965628,4.843060e-37,3.564270e-38,1.65738,139.77
791,674 Rachele (1908 EP),S,MBA,3.493765,2.351323,2.922544,0.195453,10.483907,1824.903811,1.577649e-37,1.162629e-38,1.38596,96.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,372 Palma (1893 AH),B,MBA,3.966001,2.334280,3.150141,0.258992,11.987880,2042.176736,4.065526e+18,2.615080e+17,1.44687,173.642
992,423 Diotima (1896 DB),C,MBA,3.176470,2.956412,3.066441,0.035882,11.224330,1961.328629,4.235026e+18,2.908848e+17,1.94249,175.859
993,702 Alauda (1910 KQ),B,MBA,3.247375,3.137278,3.192327,0.017244,12.757615,2083.336507,5.408989e+18,3.264933e+17,2.12080,190.98
994,334 Chicago (1892 L),C,OMB,3.981728,3.798870,3.890299,0.023502,11.703702,2802.676341,6.115258e+18,4.009727e+17,2.78295,198.77


Median profit = 0
<br> 1 = profitable
<br> 0 = not profitable

In [9]:
# Creating a row with 0 and 1 as values to be able to run machine learning models using this data
new_df['profitable'] = (new_df['profit'] > 1).astype(int)
new_df['diameter'] = new_df['diameter'].astype(float)
new_df.to_csv('asteroid_data.csv', header = True)

/var/folders/hg/1x8_3bn925l5f0m3nvml3yh40000gn/T/ipykernel_18915/2390934984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['profitable'] = (new_df['profit'] > 1).astype(int)
/var/folders/hg/1x8_3bn925l5f0m3nvml3yh40000gn/T/ipykernel_18915/2390934984.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['diameter'] = new_df['diameter'].astype(float)


# Supervised Machine Learning

In [ ]:
# Dropping non-beneficial columns
X = new_df.drop(['full_name','profit','price','spec', 'class','profitable'], axis =1)
y = new_df['profitable']

## Key Takeaway

Dropped profit and price assuming we don't already know that information about a newly discovered asteroid. Dropped spectral type since we are very unlikely to know the spectral type for a new asteroid. "The overwhelming majority of asteroids have no spectral classification and are missing other important data attributes." (asterank.com) Dropped class because class is determined by the other features. 

In [ ]:
# Converting categorical data into a numeric format
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Scaling the data for best results
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Training a Random Forest Classifier model and printing the score
clf_for = RandomForestClassifier(random_state=1, n_estimators=100).fit(X_train_scaled, y_train)

print(f'Training Score: {clf_for.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf_for.score(X_test_scaled, y_test)}')

model 1 (RandomForest) ==> Good accuracy scores, not overfitting

In [ ]:
# Creating new x_train and y_train variables based on different data
sel = SelectFromModel(clf_for)
sel.fit(X_train_scaled, y_train)

In [ ]:
# Training a Logistic Regression model and printing the score
clf_log = LogisticRegression().fit(X_train_scaled, y_train)

print(f'Training Score: {clf_log.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf_log.score(X_test_scaled, y_test)}')

In [ ]:
# Scaling and transforming new variables based on different data
X_selected_train, X_selected_test, y_train, y_test = train_test_split(sel.transform(X), y, random_state=1)
scaler = StandardScaler().fit(X_selected_train)

X_selected_train_scaled = scaler.transform(X_selected_train)
X_selected_test_scaled = scaler.transform(X_selected_test)

In [ ]:
# Training a Logistic Regression with different selected data and printing the results
clf_log_sel = LogisticRegression().fit(X_selected_train_scaled, y_train)

print(f'Training Score: {clf_log_sel.score(X_selected_train_scaled, y_train)}')
print(f'Testing Score: {clf_log_sel.score(X_selected_test_scaled, y_test)}')

model 2 (Logistic Regression) ==> decent accuracy, but training score is a bit low

In [ ]:
# Checking to see which features were most important to drop features that primarily added noise
feature_importances = clf_for.feature_importances_
features = sorted(zip(X.columns, clf_for.feature_importances_), key = lambda x: x[1])
cols = [f[0] for f in features]
width = [f[1] for f in features]


# Creating the graph to visualize the feature importance
fig, ax = plt.subplots()

fig.set_size_inches(10,10)
plt.margins(y=0.001)

ax.barh(y=cols, width=width)

plt.show()

In [ ]:
# Saving the Random Forest Classifier Model
joblib.dump(clf_for,"ml_models/rf_model.joblib")

In [ ]:
# Saving the Logistic Regression (Selected) Model
joblib.dump(clf_log_sel,"ml_models/log_sel_model.joblib")

In [ ]:
# Saving the Logistic Regression Model
joblib.dump(clf_log,"ml_models/log_model.joblib")

# Unsupervised Machine Learning

In [ ]:
# Finding which columns have null values
for column in new_df.columns:
    print(f"Column {column} has {new_df[column].isnull().sum()} null values")

In [ ]:
# Drop non beneficial columns
unsup_df = new_df.drop(['full_name'], axis = 1)
unsup_df

In [ ]:
# Convert the features into numeric values
X = pd.get_dummies(unsup_df, columns=['spec' , 'class'])
X.columns

In [ ]:
# Scale the data
ss = StandardScaler()
scaled_df = ss.fit_transform(X)

In [ ]:
# Number of features
len(X.columns)

In [ ]:
# Preserve 90% of the explained variance in dimensionality reduction.
pca = PCA(n_components = .9)

df_pca = pca.fit_transform(scaled_df)
df_pca

In [ ]:
# Convert your data into a dataframe
space_df = pd.DataFrame(df_pca)
space_df

In [ ]:
# Display the explained variance
print(pca.explained_variance_ratio_)
print("------------------------------------------------------------------")
print("  ")

# New number of features after reducing and printing finding
new_feat = len(space_df.columns)
print(f" The amount of features went from 48 to {new_feat} after perserving 90% of the explained variance in dimensionality reduction")

In [ ]:
# Further reduce the dataset dimensions with t-SNE
tsne = TSNE(learning_rate = 20)
tsne_feat = tsne.fit_transform(space_df)
tsne_feat

In [ ]:
# Visually inspect the results
space_df["x"] = tsne_feat[:,0]
space_df["y"] = tsne_feat[:,1]

# Plot 
plt.scatter(space_df["x"] , space_df["y"])
plt.savefig("images/clustered_scatter.PNG")
plt.show()
print('These clusters are from the class feature and spectral type feature that help to identify asteroids')

## Key Takeaway

Like in our unsupervised models, we need to have some kind of identifier present in our model or else we will be unable to effectively be able to categorize the asteroids in our data. This means that we must include either 'class' or 'spec' as an identifier for there to be significant findings with our data.

In [ ]:
# Create the cluster analysis with k means
inertia = []
k = list(range(1 , 11))

# Calculate the inertia for the selected range of k values
for i in k:
    km = KMeans(n_clusters = i, random_state = 42)
    km.fit(space_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k":k , "inertia": inertia}
elbow_df = pd.DataFrame(elbow_data)
print(elbow_df)
plt.plot(elbow_df['k'], elbow_df['inertia'])
plt.xticks(range(1,11))
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Curve')
plt.savefig("images/elbow_curve.PNG")
plt.show()

In [ ]:
space_df

We already knew that asteroids were grouped by class and spec, but by using this model it showed us and confirmed that we cannot remove features that help identify since all other numeric features are too close for our models to differentiate between different asteroids based on quantifiable features alone.

In [ ]:
plt.figure(figsize=(10,10))
heat_map = sns.heatmap(data = unsup_df.corr(), vmin=-1, vmax=1, annot=True, cmap = "YlGnBu")
heat_map.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12)
plt.savefig("images/heat_map.PNG")
plt.show()

Based on this heatmap above we can see there that some of the strongest correlation for profitability based on the figure is diameter. However, based on the machine learning models we know that the exclusion of spectral type (due to it being non numeric in nature) doesnt truly portray what influences profitability the most.

# Deep Learning

In [ ]:
# Drop non-beneficial columns
deep_df = new_df.drop(['full_name','spec','class'],axis=1)
deep_df

In [ ]:
# Determine the number of unique values in each column
deep_df.nunique(axis=0)

In [ ]:
# Check spec column for binning
# deep_df['spec'].value_counts()

In [ ]:
# # Grabbing the bottom 3 values and throwing them all into a single value 'other'
# spec_to_replace = ['O','R','S(IV)']

# # Replace in dataframe
# for spec in spec_to_replace:
#     deep_df['spec'] = deep_df['spec'].replace(spec,'Other')

# # Make sure binning was successful
# deep_df['spec'].value_counts()

In [ ]:
# # Check class column for binning
# deep_df['class'].value_counts()

In [ ]:
# # Grabbing the bottom 2 values and throwing them all into a single value 'other'
# class_to_replace = ['CEN','IMB']

# # Replace in dataframe
# for clas in class_to_replace:
#     deep_df['class'] = deep_df['class'].replace(clas,'Other')

# # Make sure binning was successful
# deep_df['class'].value_counts()

In [ ]:
# Create our dummy variables
dummy = pd.get_dummies(deep_df)
dummy

In [ ]:
# Split our preprocessed data into our features and target arrays
X = dummy.drop(['profitable', 'profit', 'price'],axis=1).values
y = dummy['profitable'].values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_features = len(X_train_scaled[0])
hidden_node1 = 200
hidden_node2 = 400
hidden_node3 = 30
hidden_node4 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node1, input_dim=num_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node3, activation='sigmoid'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node4, activation='sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs = 220)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
history_df = pd.DataFrame(fit_model.history)
history_df.to_hdf('ml_models/asteroid_optimizer2.h5','df')

# Plot the accuracy and loss of the model
history_df.index += 1
fig_acc = history_df.plot(y="accuracy")
fig_loss = history_df.plot(y="loss", color='red')
fig_acc.figure.savefig("images/acc_plot.PNG")
fig_loss.figure.savefig("images/loss_plot.PNG")

Based on the elbow curves above, there is a very high learning rate and also a high accuracy. This is due to the fact that spectral type is by all means the most important factor in determining if an asteroid is worth mining. It is the single most important feature for the model to work followed by class. 

In [ ]:
model = pd.read_hdf('ml_models/asteroid_optimizer2.h5') 


In [ ]:

# model = keras.models.load_model('ml_models/asteroid_optimizer2.h5')
# # model = tf.keras.Sequential([  tf.keras.layers.InputLayer(input_shape=(1,0)),  tf.keras.layers.Dense(1) ])
# test = model.predict(X_train_scaled)
rf_model = joblib.load("ml_models/rf_model.joblib")
url = "http://asterank.com/api/asterank?query=" 
num = 100
response = requests.get(url + '{"dv":{"$gt":0}}&limit=' + str(num)).json()
#print(response[0]['profit'])
asteroid_test = scaler.transform(np.array([response[0]['ad'],
                        response[0]['q'], 
                        response[0]['a'], 
                        response[0]['e'], 
                        response[0]['dv'], 
                        response[0]['per'], 
                        response[0]['moid'], 
                        response[0]['diameter']]).reshape(1,-1))

for n in range(1,num):
    asteroid_test2 = np.array([response[n]['ad'],
                        response[n]['q'], 
                        response[n]['a'], 
                        response[n]['e'], 
                        response[n]['dv'], 
                        response[n]['per'], 
                        response[n]['moid'], 
                        response[n]['diameter']]).reshape(1,-1)
    asteroid_test = np.append(asteroid_test, scaler.transform(asteroid_test2),0)
#print(asteroid_test)
test = rf_model.predict(asteroid_test)
num_accurate = 0
for n in range(num):
    print(response[n]['profit'])
    print(test[n])
    if (test[n] == 0 and response[n]['profit']<1) or (test[n] == 1 and response[n]['profit']>1):
        print('accurate')
        num_accurate +=1
    else:
        print('inaccurate')
    print("--------------------")
print(str((num_accurate/num)*100)+'% accurate')
test

In [ ]:
inputs = [{'Aphelion Distance':'input_1',
'Perihelion':'input_2',
'Semi-Major Axis':'input_3',
'Eccentricity':'input_4',
'Delta Velocity':'input_5',
'Period':'input_6',
'Minimum Orbit Intersection Distance':'input_7', 
'Diameter':'input_8',}]

In [15]:
import sqlite3
from sqlalchemy import create_engine

In [25]:
# conn = sqlite3.connect("Development_2/astroid_db.sqlite")

# cur = conn.cursor()

# for row in cur.execute('SELECT * FROM Model;'):
#     print(row)

# con.close()
database_path = "Development_2/astroid_db.sqlite"
engine = create_engine(f"sqlite:///{database_path}")

In [26]:
data = engine.execute("SELECT * FROM astroids")
for i in data:
    print(i)

(1, 3.4213, 4.4424, 3.44421, 4.3244, 2.123, 3.324, 3.453, 2.344)
(2, 3.4213, 4.4424, 3.44421, 4.3244, 2.123, 3.324, 3.43, 2.344)
(3, 3.4213, 4.4424, 3.44421, 4.3244, 2.123, 3.324, 3.43, 2.344)
(4, 3.4213, 4.4424, 3.44421, 4.344, 2.123, 3.324, 3.43, 2.344)
(5, 3.4213, 4.4424, 3.44421, 4.544, 2.123, 3.324, 3.43, 2.344)
(6, 1.1, 4.4424, 3.44421, 4.544, 2.123, 3.324, 3.43, 2.344)
(7, 1.2, 4.4424, 3.44421, 4.544, 2.123, 3.324, 3.43, 2.344)
